In [ ]:
import pandas as pd
from pathlib import Path
import sys
import IMP 
import IMP.atom
sys.path.append(str(Path(Path.home(), "Documents/mtorc2/single_traj/src")))
from analysis import get_closest_bead, is_res_10

### Check xls.

In [6]:
xl_sat_file = Path(Path.home(), "Documents/mtorc2/data/xlms/all_xl.csv")
xl_df = pd.read_csv(xl_sat_file)
xl_df = xl_df.drop(columns=["Unnamed: 0"])
xl_df.head() 


,prot1,res1,prot2,res2
0,MTOR,1197,RICTOR,516
1,MTOR,1218,RICTOR,516
2,RICTOR,1642,MSIN1,102
3,RICTOR,1092,MTOR,1993
4,RICTOR,1642,MSIN1,104


In [7]:
em_prots = ["RICTOR", "MTOR", "MLST8"]
for i in range(len(xl_df)):
    prot1, res1, prot2, res2 = tuple(xl_df.iloc[i])
        
    if (prot1 in em_prots and prot2 == "AKT1") or (prot2 in em_prots and prot1 == "AKT1"): 
        print(prot1, res1, prot2, res2) 
    if (prot1 in em_prots and prot2 == "MSIN1") or (prot2 in em_prots and prot1 == "MSIN1"): 
        print(prot1, res1, prot2, res2) 


RICTOR 1642 MSIN1 102
RICTOR 1642 MSIN1 104
AKT1 30 RICTOR 11
MSIN1 422 MLST8 215
AKT1 30 RICTOR 813
MSIN1 95 RICTOR 1614
AKT1 189 MTOR 533
RICTOR 1590 MSIN1 104
AKT1 182 RICTOR 516
RICTOR 719 AKT1 163
MTOR 1256 AKT1 420
MLST8 313 MSIN1 176
RICTOR 813 AKT1 163
MTOR 309 MSIN1 132
MSIN1 317 RICTOR 1095
AKT1 377 RICTOR 97
MTOR 42 AKT1 284
MTOR 2090 AKT1 189
AKT1 276 RICTOR 800
RICTOR 566 MSIN1 317
MSIN1 166 MLST8 307
MSIN1 132 MLST8 322
MTOR 1457 MSIN1 422
MSIN1 147 MLST8 305
RICTOR 545 AKT1 163
AKT1 322 RICTOR 11


### Get formatted XLs.

In [30]:
pdb_file = Path(Path.home(), "Documents/mtorc2/single_traj/exp_6/55/sample/output_0/pdbs/model.0.pdb")
m = IMP.Model() 
h = IMP.atom.read_pdb(str(pdb_file), m)
pids = list(m.get_particle_indexes())

atoms_dict = {"prot": [], "copy": [], "res": [], "xyz": [], "bead": []}
for pid in pids: 
    p_name = m.get_particle_name(pid)
    if IMP.atom.Residue.get_is_setup(m, pid): 
        res = IMP.atom.Residue(m, pid)
        chain_id = IMP.atom.Chain(res.get_parent()).get_id()
        prot, copy = inv_chain_ids[chain_id].split("_")

        atoms_dict["prot"].append(prot)
        atoms_dict["copy"].append(int(copy))
        atoms_dict["res"].append(int(res.get_index()))
        atoms_dict["xyz"].append(IMP.core.XYZ(res.get_child(0)).get_coordinates())
        if str(res.get_residue_type()).split("\"")[1] == "BEA": 
            atoms_dict["bead"].append(1)
        else: 
            atoms_dict["bead"].append(0)

atoms_df = pd.DataFrame(atoms_dict)
atoms_df.head(10)


,prot,copy,res,xyz,bead
0,MTOR,0,6,"[190.302001953125, 164.6179962158203, 72.49500...",1
1,MTOR,0,16,"[192.48300170898438, 173.26300048828125, 83.77...",1
2,MTOR,0,26,"[180.9929962158203, 168.47900390625, 86.411003...",1
3,MTOR,0,36,"[191.3070068359375, 159.81300354003906, 90.348...",1
4,MTOR,0,46,"[187.26300048828125, 155.9290008544922, 103.77...",1
5,MTOR,0,56,"[194.86000061035156, 154.75799560546875, 113.9...",1
6,MTOR,0,61,"[196.36199951171875, 167.91799926757812, 128.4...",0
7,MTOR,0,62,"[194.98699951171875, 167.35800170898438, 124.8...",0
8,MTOR,0,63,"[196.86000061035156, 170.2949981689453, 123.20...",0
9,MTOR,0,64,"[200.25, 168.8730010986328, 124.29399871826172]",0


In [31]:
chain_ids = dict()
chain_ids["MTOR_0"] = "A"
chain_ids["RICTOR_0"] = "B"
chain_ids["MLST8_0"] = "C"
chain_ids["MSIN1_0"] = "D" 
chain_ids["CRIM_0"] = "E"
chain_ids["RBD_0"] = "F"
chain_ids["MSIN1PH_0"] = "G"
chain_ids["AKT1PH_0"] = "H"
chain_ids["KINASE_0"] = "I"

chain_ids["MTOR_1"] = "J"
chain_ids["RICTOR_1"] = "K"
chain_ids["MLST8_1"] = "L"
chain_ids["MSIN1_1"] = "M" 
chain_ids["CRIM_1"] = "N"
chain_ids["RBD_1"] = "O"
chain_ids["MSIN1PH_1"] = "P"
chain_ids["AKT1PH_1"] = "Q"
chain_ids["KINASE_1"] = "R"


In [1]:
a_xls = ["AKT1 30 - RICTOR 11", "AKT1 322 - RICTOR 11", "AKT1 377 - RICTOR 97", \
 "MSIN1 95 - RICTOR 1614", "MSIN1 102 - RICTOR 1642", "MSIN1 104 - RICTOR 1642", \
 "MSIN1 104 - RICTOR 1590", "MSIN1 132 - MLST8 322", "MSIN1 147 - MLST8 305", \
 "MSIN1 166 - MLST8 307", "MSIN1 176 - MLST8 313", "MSIN1 422 - MTOR 1457", \
 "MSIN1 422 - MLST8 215", "MSIN1 422 - AKT1 170"]
b_xls = ["AKT1 30 - RICTOR 813", "AKT1 163 - RICTOR 719", \
 "AKT1 163  - RICTOR 813", "AKT1 163 - RICTOR 545", "AKT1 182 - RICTOR 516", \
 "AKT1 189 - MTOR 533", "AKT1 189 - MTOR 2090", "AKT1 276 - RICTOR 800", \
 "AKT1 284 - MTOR 42", "AKT1 420 - MTOR 1256", "MSIN1 132 - MTOR 309", \
 "MSIN1 317 - RICTOR 566", "MSIN1 317 - RICTOR 1095"]

In [44]:
a_xl_dict = {"prot1": list(), "res1": list(), "prot2": list(), "res2": list()}
b_xl_dict = {"prot1": list(), "res1": list(), "prot2": list(), "res2": list()}

In [59]:
is_b = False 
labels = ""
for xls in [a_xls, b_xls]:
    for i in range(len(xls)):
        xl = xls[i]
        prot1, res1 = xl.split("-")[0].split(" ")[0], int(xl.split("-")[0].split(" ")[1])
        prot2, res2 = xl.split("-")[1].split(" ")[1], int(xl.split("-")[1].split(" ")[2])
        orig_prot1, orig_res1, orig_prot2, orig_res2 = prot1, res1, prot2, res2
        
        if prot1 == "MSIN1" and (res1 >= 158 and res1 < 280): 
            prot1 = "CRIM"
            res1 = res1-158+1 
        if prot2 == "MSIN1" and (res2 >= 158 and res2 < 280): 
            prot2 = "CRIM"
            res2 = res2-158+1   

        if prot1 == "MSIN1" and (res1 >= 280 and res1 < 382): 
            prot1 = "RBD"
            res1 = res1-280+1
        if prot2 == "MSIN1" and (res2 >= 280 and res2 < 382): 
            prot2 = "RBD"
            res2 = res2-280+1

        if prot1 == "MSIN1" and (res1 >= 382): 
            prot1 = "MSIN1PH"
            res1 = res1-382+1
        if prot2 == "MSIN1" and (res2 >= 382): 
            prot2 = "MSIN1PH"
            res2 = res2-382+1

        if prot1 == "AKT1" and res1 < 146: 
            prot1 = "AKT1PH"
        if prot2 == "AKT1" and res2 < 146: 
            prot2 = "AKT1PH"

        if prot1 == "AKT1" and res1 >= 146: 
            prot1 = "KINASE"
            res1 = res1-146+1 
        if prot2 == "AKT1" and res2 >= 146: 
            prot2 = "KINASE"
            res2 = res2-146+1  
            
        if is_b:
            b_xl_dict["prot1"].append(prot1), b_xl_dict["prot2"].append(prot2)
            b_xl_dict["res1"].append(res1), b_xl_dict["res2"].append(res2)
        else: 
            a_xl_dict["prot1"].append(prot1), a_xl_dict["prot2"].append(prot2)
            a_xl_dict["res1"].append(res1), a_xl_dict["res2"].append(res2)
            
        if is_res_10(atoms_df, prot1, res1, 0): 
            res1 = get_closest_bead(atoms_df, prot1, res1, 0)
        if is_res_10(atoms_df, prot2, res2, 0):
            res2 = get_closest_bead(atoms_df, prot2, res2, 0)
        
        chain_1, chain_2 = chain_ids[prot1+"_0"], chain_ids[prot2+"_0"]
        print("#1/{}:{}@CA #1/{}:{}@CA".format(chain_1, res1, chain_2, res2))
        label = 'label #1/{}:{}/{}:{} pseudobonds text "{} {} - {} {}" size 100 height 2\n'.format(chain_1, res1, chain_2, res2, orig_prot1, orig_res1, orig_prot2, orig_res2)                                                                    
        labels = labels + label
            
    print("FIN")        
    is_b = True 



#1/H:30@CA #1/B:16@CA
#1/I:177@CA #1/B:16@CA
#1/I:232@CA #1/B:97@CA
#1/D:95@CA #1/B:1614@CA
#1/D:102@CA #1/B:1642@CA
#1/D:104@CA #1/B:1642@CA
#1/D:104@CA #1/B:1595@CA
#1/D:132@CA #1/C:322@CA
#1/D:148@CA #1/C:305@CA
#1/E:9@CA #1/C:307@CA
#1/E:19@CA #1/C:313@CA
#1/G:38@CA #1/A:1457@CA
#1/G:38@CA #1/C:215@CA
FIN
#1/H:30@CA #1/B:813@CA
#1/I:18@CA #1/B:719@CA
#1/I:18@CA #1/B:813@CA
#1/I:18@CA #1/B:545@CA
#1/I:37@CA #1/B:518@CA
#1/I:44@CA #1/A:533@CA
#1/I:44@CA #1/A:2090@CA
#1/I:131@CA #1/B:800@CA
#1/I:139@CA #1/A:46@CA
#1/I:275@CA #1/A:1257@CA
#1/D:132@CA #1/A:305@CA
#1/F:38@CA #1/B:566@CA
#1/F:38@CA #1/B:1091@CA
FIN


In [60]:
print(labels)

label #1/H:30/B:16 pseudobonds text "AKT1 30 - RICTOR 11" size 100 height 2
label #1/I:177/B:16 pseudobonds text "AKT1 322 - RICTOR 11" size 100 height 2
label #1/I:232/B:97 pseudobonds text "AKT1 377 - RICTOR 97" size 100 height 2
label #1/D:95/B:1614 pseudobonds text "MSIN1 95 - RICTOR 1614" size 100 height 2
label #1/D:102/B:1642 pseudobonds text "MSIN1 102 - RICTOR 1642" size 100 height 2
label #1/D:104/B:1642 pseudobonds text "MSIN1 104 - RICTOR 1642" size 100 height 2
label #1/D:104/B:1595 pseudobonds text "MSIN1 104 - RICTOR 1590" size 100 height 2
label #1/D:132/C:322 pseudobonds text "MSIN1 132 - MLST8 322" size 100 height 2
label #1/D:148/C:305 pseudobonds text "MSIN1 147 - MLST8 305" size 100 height 2
label #1/E:9/C:307 pseudobonds text "MSIN1 166 - MLST8 307" size 100 height 2
label #1/E:19/C:313 pseudobonds text "MSIN1 176 - MLST8 313" size 100 height 2
label #1/G:38/A:1457 pseudobonds text "MSIN1 422 - MTOR 1457" size 100 height 2
label #1/G:38/C:215 pseudobonds text "MSI

In [47]:
a_xl_df = pd.DataFrame(a_xl_dict)
a_xl_df.to_csv(Path(Path.home(), "Documents/mtorc2/data/xlms/A.csv"))
a_xl_df.head()

,prot1,res1,prot2,res2
0,AKT1PH,30,RICTOR,11
1,KINASE,177,RICTOR,11
2,KINASE,232,RICTOR,97
3,MSIN1,95,RICTOR,1614
4,MSIN1,102,RICTOR,1642


In [48]:
b_xl_df = pd.DataFrame(b_xl_dict)
b_xl_df.to_csv(Path(Path.home(), "Documents/mtorc2/data/xlms/B.csv"))
b_xl_df.head()

,prot1,res1,prot2,res2
0,AKT1PH,30,RICTOR,813
1,KINASE,18,RICTOR,719
2,KINASE,18,RICTOR,813
3,KINASE,18,RICTOR,545
4,KINASE,37,RICTOR,516
